In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np

In [10]:
def get_carbon_data(url):
    url = f"{url}"
    headers = {
    'x-rapidapi-host': "daily-atmosphere-carbon-dioxide-concentration.p.rapidapi.com",
    'x-rapidapi-key': "6f91c81d8fmshc83b86b1b6c6c5cp108956jsnef221ca08c9a"
    }
    response = requests.request("GET", url, headers=headers)
    t = response.json()
    return t

In [85]:
c = pd.DataFrame(t['co2'])

In [91]:
c.tail(5)

,year,month,day,cycle,trend
3887,2021,8,23,411.45,415.31
3888,2021,8,24,411.45,415.31
3889,2021,8,25,411.45,415.32
3890,2021,8,26,411.45,415.33
3891,2021,8,27,411.46,415.33


#### Creamos cd con todos los años para analizar dsp

In [60]:
def cod_agrupado(x):
    return x.groupby(["year", "month"]).agg({"cycle": "median"})

In [61]:
cd = cod_agrupado(c)

In [62]:
cd.head(3)

cycle
year month        
2011 1      391.68
     10     389.47
     11     391.38

#### Escalamos a cd para que tenga solo los años que nos interesan

In [95]:
dates = pd.period_range(start='2011-01-01', end='2021-08-27', freq='D')

In [97]:
c['Date'] = dates

In [99]:
display(c)

,year,month,day,cycle,trend,Date
0,2011,1,1,391.26,389.73,2011-01-01
1,2011,1,2,391.29,389.73,2011-01-02
2,2011,1,3,391.33,389.74,2011-01-03
3,2011,1,4,391.36,389.75,2011-01-04
4,2011,1,5,391.39,389.75,2011-01-05
...,...,...,...,...,...,...
3887,2021,8,23,411.45,415.31,2021-08-23
3888,2021,8,24,411.45,415.31,2021-08-24
3889,2021,8,25,411.45,415.32,2021-08-25
3890,2021,8,26,411.45,415.33,2021-08-26


In [26]:
te = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/temperature.csv')

In [27]:
te.columns

Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa',
       'Nahariyya', 'Jerusalem'],
      dtype='object')

In [29]:
te.shape

(45253, 37)

In [30]:
nulls = [te[i].isna().value_counts() for i in te.columns]

In [31]:
print(nulls[2])

False    45252
True         1
Name: Portland, dtype: int64


### eliminamos columnas inecesarias (solo usa y canada)

In [32]:
def keep_columns(x, y):
    """
    Toma como argumento la lista de las columnas que quiero conservar y el nombre del data frame.
    Recorre el indice de columnas del data frame correspondiente y elimina las
    columnas que no quiero conservar
    """
    relevant = x
    for i in y.columns:
        if i not in relevant:
            y.drop(columns=[i], inplace=True)

In [33]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], te)

### Agrupamos por fecha y mediana

In [34]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").median()
    

In [35]:
tem = mediana_mensual(te)

In [77]:
tem.head(3)

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
Date,,,,,,,,,,,,,,,,,,,,,
2012-10,10.050,11.630,15.730,10.630,19.260,18.85,20.35,23.740,13.83,9.730,...,11.370,23.24,15.505086,26.25,11.2400,10.11,14.190,14.750,10.600,13.150
2012-11,7.165,8.405,13.510,7.900,15.500,15.52,14.32,17.705,7.45,7.190,...,4.010,14.86,8.470000,21.66,3.4900,3.55,5.420,5.695,1.255,4.335
2012-12,4.025,5.110,10.665,5.065,13.245,13.13,8.82,11.405,1.31,0.275,...,2.455,15.96,9.195000,22.19,3.0025,0.75,4.575,4.490,-3.135,1.935


### transformamos a celcius

In [37]:
def celsius_conv(row):
    for i in tem.index:
           row[i] = row[i] - 273.15
    return row
          

temc = tem.apply(lambda row : celsius_conv(row))

In [38]:
temc.head()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
Date,,,,,,,,,,,,,,,,,,,,,
2012-10,10.050,11.630,15.730,10.630,19.260,18.85,20.350,23.740,13.83000,9.730,...,11.370,23.24,15.505086,26.250000,11.2400,10.110,14.190,14.7500,10.600000,13.150
2012-11,7.165,8.405,13.510,7.900,15.500,15.52,14.320,17.705,7.45000,7.190,...,4.010,14.86,8.470000,21.660000,3.4900,3.550,5.420,5.6950,1.255000,4.335
2012-12,4.025,5.110,10.665,5.065,13.245,13.13,8.820,11.405,1.31000,0.275,...,2.455,15.96,9.195000,22.190000,3.0025,0.750,4.575,4.4900,-3.135000,1.935
2013-01,2.410,1.535,8.360,2.810,11.585,11.66,5.705,9.310,-1.58875,-0.535,...,-1.540,15.96,7.640000,22.485000,-0.6800,-1.110,1.985,2.0150,-8.603625,-0.455
2013-02,5.190,5.990,9.170,6.120,12.135,11.69,9.380,11.350,2.81000,-1.660,...,-2.555,16.15,5.425000,21.997867,-1.9850,-3.645,0.235,0.1725,-7.500000,-1.080


In [39]:
temc.shape

(62, 30)

In [40]:
h = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/humidity.csv')

In [41]:
h.head(3)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0
2,2012-10-01 14:00:00,76.0,80.0,87.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,91.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0


In [42]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], h)

In [43]:
h.columns

Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'],
      dtype='object')

In [44]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").mean()

In [45]:
hm = mediana_mensual(h)

In [ ]:
tecm['Carbon Levels'] = 

In [ ]:
for i in range(365):
    c.drop([i], axis=0, inplace=True)